**Data Pre-Processing**

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from glob import glob
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_images_path = "/content/drive/MyDrive/SignLanguage/TurkishSignLanguageYoloV8/train/images"
train_labels_path = "/content/drive/MyDrive/SignLanguage/TurkishSignLanguageYoloV8/train/labels"

In [ ]:
#normalizing the image dimensions:RESIZE And NORMALIZE

def resize_and_normalize_images(image_folder, target_size=(640, 640)):
    resized_folder = f"{image_folder}_resized"
    os.makedirs(resized_folder, exist_ok=True)

    for img_path in tqdm(glob(os.path.join(image_folder, "*.jpg"))):

        img = cv2.imread(img_path)

        resized_img = cv2.resize(img, target_size)

        normalized_img = resized_img / 255.0


        new_img_path = os.path.join(resized_folder, os.path.basename(img_path))
        cv2.imwrite(new_img_path, (normalized_img * 255).astype(np.uint8))

    return resized_folder

In [ ]:
#Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_images(image_folder, save_to_folder, augment_count=3):
    os.makedirs(save_to_folder, exist_ok=True)
    #Data Generator
    datagen = ImageDataGenerator(
        rotation_range=30,  #Rotation Range
        width_shift_range=0.2,  #Width Shift
        height_shift_range=0.2,  #Height Shift
        shear_range=0.2,  #Shear Range
        zoom_range=0.2,  #Zoom Range
        horizontal_flip=True,  #Horizontal
        fill_mode='nearest'  #Fill Mode
    )

    for img_path in tqdm(glob(os.path.join(image_folder, "*.jpg"))):
        img = cv2.imread(img_path)
        # make the appropriate shape
        img = img.reshape((1,) + img.shape)
        i = 0
        # Creating a certain number of augmented versions for each image
        for batch in datagen.flow(img, batch_size=1, save_to_dir=save_to_folder, save_prefix='aug', save_format='jpg'):
            i += 1
            if i >= augment_count:
                break

In [ ]:
#Applying Resize and Normalize Function
resized_images_folder = resize_and_normalize_images(train_images_path)

100%|██████████| 20043/20043 [15:12<00:00, 21.97it/s]


In [ ]:
#Applying Augmentation Function
augment_save_folder = "/content/drive/MyDrive/SignLanguage/TurkishSignLanguageYoloV8/train/augmented_images"
augment_images(resized_images_folder, augment_save_folder)

100%|██████████| 20043/20043 [1:47:52<00:00,  3.10it/s]
